# 10 minutes to `pandas`

This is a short introduction to pandas, geared mainly for new users and adapted to the Jupyter notebook format from the [`pandas` documentation](https://pandas.pydata.org/docs/user_guide/10min.html). You
can see more complex recipes in the [Cookbook](https://pandas.pydata.org/docs/user_guide/cookbook.html#cookbook).

Customarily, we import as follows:

In [ ]:
import numpy as np
import pandas as pd

## Object creation

See the [Data Structure Intro section](https://pandas.pydata.org/docs/user_guide/dsintro.html#dsintro).

Creating a `Series` by passing a list of values, letting pandas create a
default integer index:


In [ ]:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s


Creating a `DataFrame` by passing a NumPy array, with a datetime index
and labeled columns:



In [ ]:
dates = pd.date_range("20130101", periods=6)
dates

In [ ]:
df = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list("ABCD"))
df


Creating a `DataFrame` by passing a dict of objects that can be
converted to series-like.



In [ ]:
df2 = pd.DataFrame({  
  "A": 1.0, 
  "B": pd.Timestamp("20130102"),
  "C": pd.Series(1,index=list(range(4)), dtype="float32"), 
  "D": np.array([3] * 4, dtype="int32"),
  "E": pd.Categorical(["test", "train", "test", "train"]), 
  "F": "foo",
})
df2

The columns of the resulting `DataFrame` have different
`dtypes <basics.dtypes>`.



In [ ]:
df2.dtypes

## Viewing data

Here is how to view the top and bottom rows of the frame:


In [ ]:
df.head()
df.tail(3)


Display the index, columns:



In [ ]:
df.index
df.columns


`DataFrame.to_numpy` gives a NumPy representation of the underlying
data. Note that this can be an expensive operation when your `DataFrame`
has columns with different data types, which comes down to a fundamental
difference between pandas and NumPy: **NumPy arrays have one dtype for
the entire array, while pandas DataFrames have one dtype per column**.
When you call `DataFrame.to_numpy`, pandas will find the NumPy dtype
that can hold *all* of the dtypes in the DataFrame. This may end up
being `object`, which requires casting every value to a Python object.

For `df`, our `DataFrame` of all floating-point values,
`DataFrame.to_numpy` is fast and doesn't require copying data.



In [ ]:
df.to_numpy()


For `df2`, the `DataFrame` with multiple dtypes, `DataFrame.to_numpy` is
relatively expensive.



In [ ]:
df2.to_numpy()

Note: `DataFrame.to_numpy` does *not* include the index or column labels in
the output.

`DataFrame.describe` shows a quick statistic summary of your data:



In [ ]:
df.describe()


Transposing your data:



In [ ]:
df.T

Sorting by an axis:



In [ ]:
df.sort_index(axis=1, ascending=False)


Sorting by values:



In [ ]:
df.sort_values(by="B")

## Selection

Note: While standard Python / NumPy expressions for selecting and setting are
intuitive and come in handy for interactive work, for production code,
we recommend the optimized pandas data access methods, `.at`, `.iat`,
`.loc` and `.iloc`.

See the indexing documentation [Indexing and Selecting
Data](https://pandas.pydata.org/docs/user_guide/indexing.html) and `MultiIndex / Advanced Indexing <advanced>`.


### Getting

Selecting a single column, which yields a `Series`, equivalent to `df.A`
:



In [ ]:
df["A"]


Selecting via `[]`, which slices the rows.



In [ ]:
df[0:3]
df["20130102":"20130104"]

### Selection by label

See more in [Selection by Label](https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-label).

For getting a cross section using a label:

In [ ]:
df.loc[dates[0]]

Selecting on a multi-axis by label:



In [ ]:
df.loc[:, ["A", "B"]]

Showing label slicing, both endpoints are *included*:



In [ ]:
df.loc["20130102":"20130104", ["A", "B"]]


Reduction in the dimensions of the returned object:



In [ ]:
df.loc["20130102", ["A", "B"]]

For getting a scalar value:



In [ ]:
df.loc[dates[0], "A"]

For getting fast access to a scalar (equivalent to the prior method):



In [ ]:
df.at[dates[0], "A"]

### Selection by position

See more in [Selection by Position](https://pandas.pydata.org/docs/user_guide/10min.html#selection-by-position).

Select via the position of the passed integers:

In [ ]:
df.iloc[3]


By integer slices, acting similar to NumPy/Python:



In [ ]:
df.iloc[3:5, 0:2]


By lists of integer position locations, similar to the NumPy/Python
style:



In [ ]:
df.iloc[[1, 2, 4], [0, 2]]

For slicing rows explicitly:



In [ ]:
df.iloc[1:3, :]

For slicing columns explicitly:



In [ ]:
df.iloc[:, 1:3]


For getting a value explicitly:



In [ ]:
df.iloc[1, 1]

For getting fast access to a scalar (equivalent to the prior method):



In [ ]:
df.iat[1, 1]

### Boolean indexing

Using a single column's values to select data.



In [ ]:
df[df["A"] > 0]

Selecting values from a DataFrame where a boolean condition is met.



In [ ]:
df[df > 0]


Using the `Series.isin` method for filtering:



In [ ]:
df2 = df.copy() 
df2["E"] = ["one", "one", "two", "three", "four", "three"] 
df2 

In [ ]:
df2[df2["E"].isin(["two", "four"])]


### Setting

Setting a new column automatically aligns the data by the indexes.



In [ ]:
s1 = pd.Series([1, 2, 3, 4, 5, 6], index=pd.date_range("20130102", periods=6)) 
s1

In [ ]:
df["F"] = s1


</div>

Setting values by label:



In [ ]:
df.at[dates[0], "A"] = 0

Setting values by position:



In [ ]:
df.iat[0, 1] = 0


Setting by assigning with a NumPy array:



In [ ]:
df.loc[:, "D"] = np.array([5] * len(df))


The result of the prior setting operations.



In [ ]:
df

A `where` operation with setting.

In [ ]:
df2 = df.copy()
df2[df2 > 0] = -df2
df2

## Missing data

pandas primarily uses the value `np.nan` to represent missing data. It
is by default not included in computations. See the
[Missing Data section](https://pandas.pydata.org/docs/user_guide/missing_data.html#missing-data).

Reindexing allows you to change/add/delete the index on a specified
axis. This returns a copy of the data.

In [ ]:
df1 = df.reindex(index=dates[0:4], columns=list(df.columns) + ["E"])
df1.loc[dates[0] : dates[1], "E"] = 1
df1

To drop any rows that have missing data.

In [ ]:
df1.dropna(how="any")

Filling missing data.

In [ ]:
df1.fillna(value=5)


To get the boolean mask where values are `nan`.

In [ ]:
pd.isna(df1)


## Operations

See the `Basic section on Binary Ops <basics.binop>`.

### Stats

Operations in general *exclude* missing data.

Performing a descriptive statistic:



In [ ]:
df.mean()

Same operation on the other axis:



In [ ]:
df.mean(1)


Operating with objects that have different dimensionality and need
alignment. In addition, pandas automatically broadcasts along the
specified dimension.



In [ ]:
s = pd.Series([1, 3, 5, np.nan, 6, 8], index=dates).shift(2)
s

In [ ]:
df.sub(s, axis="index")


### Apply

Applying functions to the data:



In [ ]:
df.apply(np.cumsum)
df.apply(lambda x: x.max() - x.min())

### Histogramming

See more at [Histogramming and Discretization](https://pandas.pydata.org/docs/user_guide/basics.html#basics-discretization).

In [ ]:
s = pd.Series(np.random.randint(0, 7, size=10))
s

In [ ]:
s.value_counts()

### String Methods

Series is equipped with a set of string processing methods in the `str`
attribute that make it easy to operate on each element of the array, as
in the code snippet below. Note that pattern-matching in `str` generally
uses [regular expressions](https://docs.python.org/3/library/re.html) by
default (and in some cases always uses them). See more at
[Vectorized String Methods](https://pandas.pydata.org/docs/user_guide/text.html#text-string-methods).

In [ ]:
s = pd.Series(["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"])
s.str.lower()

## Merge

### Concat

pandas provides various facilities for easily combining together Series
and DataFrame objects with various kinds of set logic for the indexes
and relational algebra functionality in the case of join / merge-type
operations.

See the [Merging section](https://pandas.pydata.org/docs/user_guide/merging.html#merging).

Concatenating pandas objects together with `concat` :



In [ ]:
df = pd.DataFrame(np.random.randn(10, 4)) 
df

# break it into pieces 
pieces = [df[:3], df[3:7], df[7:]]

pd.concat(pieces)

Note: Adding a column to a `DataFrame` is relatively fast. However, adding a
row requires a copy, and may be expensive. We recommend passing a
pre-built list of records to the `DataFrame` constructor instead of
building a `DataFrame` by iteratively appending records to it. See
[Appending to dataframe](https://pandas.pydata.org/docs/user_guide/merging.html#merging-concatenation) for more.

### Join

SQL style merges. See the [Database style joining](https://pandas.pydata.org/docs/user_guide/merging.html#merging-join)
section.

In [ ]:
left = pd.DataFrame({"key": ["foo", "foo"], "lval": [1, 2]})
right = pd.DataFrame({"key": ["foo", "foo"], "rval": [4, 5]})
left

In [ ]:
right

In [ ]:
pd.merge(left, right, on="key")


Another example that can be given is:



In [ ]:
left = pd.DataFrame({"key": ["foo", "bar"], "lval": [1, 2]})
right = pd.DataFrame({"key": ["foo", "bar"], "rval": [4, 5]})

In [ ]:
left

In [ ]:
right

In [ ]:
pd.merge(left, right, on="key")

## Grouping

By "group by" we are referring to a process involving one or more of the
following steps:

> -   **Splitting** the data into groups based on some criteria
> -   **Applying** a function to each group independently
> -   **Combining** the results into a data structure

See the [Grouping section](https://pandas.pydata.org/docs/user_guide/groupby.html#groupby).

In [ ]:
df = pd.DataFrame({  
  "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
  "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
  "C": np.random.randn(8), "D": np.random.randn(8),
})
df


Grouping and then applying the `~pandas.core.groupby.GroupBy.sum`
function to the resulting groups.



In [ ]:
df.groupby("A").sum()

Grouping by multiple columns forms a hierarchical index, and again we
can apply the `~pandas.core.groupby.GroupBy.sum` function.



In [ ]:
df.groupby(["A", "B"]).sum()

## Reshaping

See the sections on [Hierarchical Indexing](https://pandas.pydata.org/docs/user_guide/advanced.html#advanced-hierarchical) and
[Reshaping](https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-stacking).


### Stack



In [ ]:
tuples = list(
  zip(*[
    ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
    ["one", "two", "one", "two", "one", "two", "one", "two"],
  ])
)
index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A",
"B"])
df2 = df[:4]
df2

The `~DataFrame.stack` method "compresses" a level in the DataFrame's
columns.



In [ ]:
stacked = df2.stack()
stacked


With a "stacked" DataFrame or Series (having a `MultiIndex` as the
`index`), the inverse operation of `~DataFrame.stack` is
`~DataFrame.unstack`, which by default unstacks the **last level**:



In [ ]:
stacked.unstack()
stacked.unstack(1)
stacked.unstack(0)

### Pivot tables

See the section on [Pivot Tables](https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-pivot).

In [ ]:
df = pd.DataFrame(  
  {  
  "A": ["one", "one", "two", "three"] * 3, "B": ["A", "B", "C"] * 4,
  "C": ["foo", "foo", "foo", "bar", "bar", "bar"] * 2, "D":
  np.random.randn(12), "E": np.random.randn(12),

  }
)
df


We can produce pivot tables from this data very easily:



In [ ]:
pd.pivot_table(df, values="D", index=["A", "B"], columns=["C"])

## Time series

pandas has simple, powerful, and efficient functionality for performing
resampling operations during frequency conversion (e.g., converting
secondly data into 5-minutely data). This is extremely common in, but
not limited to, financial applications. See the [Time Series
section](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries).

In [ ]:
rng = pd.date_range("1/1/2012", periods=100, freq="S")
ts = pd.Series(np.random.randint(0, 500, len(rng)), index=rng)
ts.resample("5Min").sum()


Time zone representation:



In [ ]:
rng = pd.date_range("3/6/2012 00:00", periods=5, freq="D")
ts = pd.Series(np.random.randn(len(rng)), rng)
ts

In [ ]:
ts_utc = ts.tz_localize("UTC")
ts_utc


Converting to another time zone:



In [ ]:
ts_utc.tz_convert("US/Eastern")


Converting between time span representations:



In [ ]:
rng = pd.date_range("1/1/2012", periods=5, freq="M")
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

In [ ]:
ps = ts.to_period()
ps
ps.to_timestamp()


Converting between period and timestamp enables some convenient
arithmetic functions to be used. In the following example, we convert a
quarterly frequency with year ending in November to 9am of the end of
the month following the quarter end:



In [ ]:
prng = pd.period_range("1990Q1", "2000Q4", freq="Q-NOV")
ts = pd.Series(np.random.randn(len(prng)), prng)
ts.index = (prng.asfreq("M", "e") + 1).asfreq("H", "s") + 9
ts.head()

## Categoricals

pandas can include categorical data in a `DataFrame`. For full docs, see
the [categorical introduction](https://pandas.pydata.org/docs/user_guide/categorical.html#categorical) and the
[API documentation](https://pandas.pydata.org/docs/reference/arrays.html#api-arrays-categorical).

In [ ]:
df = pd.DataFrame(
  {
    "id": [1, 2, 3, 4, 5, 6],
    "raw_grade": ["a", "b", "b", "a", "a", "e"]
  }
)

Convert the raw grades to a categorical data type.

In [ ]:
df["grade"] = df["raw_grade"].astype("category")
df["grade"]

Rename the categories to more meaningful names (assigning to
`Series.cat.categories` is in place!).

In [ ]:
df["grade"].cat.categories = ["very good", "good", "very bad"]


Reorder the categories and simultaneously add the missing categories
(methods under `Series.cat` return a new `Series` by default).


In [ ]:
df["grade"] = df["grade"].cat.set_categories(["very bad", "bad", "medium", "good", "very good"])
df["grade"]

Sorting is per order in the categories, not lexical order.


In [ ]:
df.sort_values(by="grade")

Grouping by a categorical column also shows empty categories.


In [ ]:
df.groupby("grade").size()